In [74]:
from segment import segment
from combine import combine
from PIL import Image,ImageFilter
import numpy as np
import matplotlib.pyplot as plt
from scipy import ndimage
import math



In [75]:
img=Image.open(r"images\9.png")
segmented=segment(img)
combined=combine(img,segmented)
combined.save("img.png")

In [81]:
import cv2

# Load the image
#image = cv2.imread('2.png', cv2.IMREAD_GRAYSCALE)
image=np.array(combined.convert("L"))
image_color=np.array(combined)
# Threshold the image to get a binary image
_, binary = cv2.threshold(image, 40, 255, cv2.THRESH_BINARY)

# Find contours
contours, _ = cv2.findContours(binary, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
# Fit ellipses
ellipses=[]
for contour in contours:
    if len(contour)<10:# Remove Crap
        continue
    ellipse = cv2.fitEllipse(contour)
    if sum(ellipse[1])<300:# Remove Crap
        continue
    ellipses.append(ellipse)


#remove overlaying ellipses
seens=[]
for ellipse in ellipses:
    for i,seen in enumerate(seens):
        if math.dist(ellipse[0], seen[0]) <max(seen[1][0],ellipse[1][0]):
            if sum(seen[1])>sum(ellipse[1]):
                break
            seens[i]=ellipse
    else:
        seens.append(ellipse)
ellipses=seens


for ellipse in ellipses:
    # Draw the ellipse on the image
    cv2.ellipse(image_color, ellipse, (255, 0, 0), 5)
    print(ellipse)

# Save or display the result
cv2.imwrite('fitted_ellipse.png', image_color)


((1615.034423828125, 1232.6785888671875), (389.93463134765625, 395.1799621582031), 29.259113311767578)
((400.2854919433594, 1444.3726806640625), (523.5407104492188, 1244.692138671875), 175.4304656982422)
((1571.4539794921875, 561.4215087890625), (286.3486633300781, 677.9213256835938), 134.27557373046875)


True

In [77]:
areas,num=ndimage.label(np.array(segmented).astype(bool))
all_edges=[]
for i in range(1,num+1):
    
    coords = np.argwhere(areas == i)
    y_min, x_min = coords.min(axis=0)
    y_max, x_max = coords.max(axis=0) + 1 # plus 1 to include the max range
    img_arr=np.array(img)
    img_arr[areas!=i]=0
    segmented_=np.array(segmented)
    segmented_[areas!=i]=0
    edges=np.argwhere(np.array(Image.fromarray(segmented_).filter(ImageFilter.FIND_EDGES)).astype(bool))
    all_edges.append(edges)
    out_img=Image.fromarray(img_arr)
    
    out_img =out_img.crop((x_min, y_min, x_max, y_max))
    out_img.save(f"{i}.png")